# Machine Translation English-German Example Using SageMaker Seq2Seq

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Download dataset and preprocess](#Download-dataset-and-preprocess)
3. [Training the Machine Translation model](#Training-the-Machine-Translation-model)
4. [Inference](#Inference)

## Introduction

Welcome to our Machine Translation end-to-end example! In this demo, we will train a English-German translation model and will test the predictions on a few examples.

SageMaker Seq2Seq algorithm is built on top of [Sockeye](https://github.com/awslabs/sockeye), a sequence-to-sequence framework for Neural Machine Translation based on MXNet. SageMaker Seq2Seq implements state-of-the-art encoder-decoder architectures which can also be used for tasks like Abstractive Summarization in addition to Machine Translation.

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.

## Setup

Let's start by specifying:
- The S3 bucket and prefix that you want to use for training and model data. **This should be within the same region as the Notebook Instance, training, and hosting.**
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp in the cell below with a the appropriate full IAM role arn string(s).

In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

# S3 bucket and prefix
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-seq2seq"

In [2]:
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [3]:
from time import gmtime, strftime
import time
import numpy as np
import os
import json

# For plotting attention matrix later on
import matplotlib

%matplotlib inline
import matplotlib.pyplot as plt

## Download dataset and preprocess

In this notebook, we will train a English to German translation model on a dataset from the
[Conference on Machine Translation (WMT) 2017](http://www.statmt.org/wmt17/).

In [4]:
%%bash
wget http://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.de.gz & \
wget http://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.en.gz & wait
gunzip corpus.tc.de.gz & \
gunzip corpus.tc.en.gz & wait
mkdir validation
wget http://data.statmt.org/wmt17/translation-task/preprocessed/de-en/dev.tgz
tar xzf dev.tgz -C validation

--2023-06-08 07:21:42--  http://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.de.gz
--2023-06-08 07:21:42--  http://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.en.gz
Resolving data.statmt.org (data.statmt.org)... Resolving data.statmt.org (data.statmt.org)... 129.215.197.184129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... 
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.de.gz [following]
--2023-06-08 07:21:42--  https://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.de.gz
301 Moved Permanently
Location: https://data.statmt.org/wmt17/translation-task/preprocessed/de-en/corpus.tc.en.gz [following]
--2023-06-08 07:21:42--  https://data.statmt.org/wmt17/tran

  3650K .......... .......... .......... .......... ..........  1%  250M 96s
  3700K .......... .......... .......... .......... ..........  1%  141M 94s
  3750K .......... .......... .......... .......... ..........  1%  247M 93s
  3800K .......... .......... .......... .......... ..........  1%  228M 92s
  3850K .......... .......... .......... .......... ..........  1%  247M 91s
  3900K .......... .......... .......... .......... ..........  1%  260M 89s
  3950K .......... .......... .......... .......... ..........  1%  256M 88s
  4000K .......... .......... .......... .......... ..........  1%  212M 87s
  4050K .......... .......... .......... .......... ..........  1%  260M 86s
  4100K .......... .......... .......... .......... ..........  1%  200M 85s
  4150K .......... .......... .......... .......... ..........  1% 13.5M 84s
  4200K .......... .......... .......... .......... ..........  1% 24.7M 83s
  4250K .......... .......... .......... .......... ..........  1% 56.7M 82s

  7600K .......... .......... .......... .......... ..........  2%  110M 52s
  7650K .......... .......... .......... .......... ..........  2%  261M 52s
  7700K .......... .......... .......... .......... ..........  3%  241M 51s
  7750K .......... .......... .......... .......... ..........  3%  515K 54s
  7800K .......... .......... .......... .......... ..........  3% 47.9M 54s
  7850K .......... .......... .......... .......... ..........  3% 31.9M 53s
  7900K .......... .......... .......... .......... ..........  3%  182M 53s
  7950K .......... .......... .......... .......... ..........  3%  233M 53s
  8000K .......... .......... .......... .......... ..........  3%  180M 52s
  8050K .......... .......... .......... .......... ..........  3%  226M 52s
  8100K .......... .......... .......... .......... ..........  3%  225M 52s
  8150K .......... .......... .......... .......... ..........  3% 37.8M 51s
  8200K .......... .......... .......... .......... ..........  3%  201M 51s

 11100K .......... .......... .......... .......... ..........  4% 88.0M 41s
 11150K .......... .......... .......... .......... ..........  4% 81.6M 41s
 11200K .......... .......... .......... .......... ..........  4% 77.7M 41s
 11250K .......... .......... .......... .......... ..........  4% 79.2M 41s
 11300K .......... .......... .......... .......... ..........  4% 96.5M 40s
 11350K .......... .......... .......... .......... ..........  4% 80.7M 40s
 11400K .......... .......... .......... .......... ..........  4% 87.8M 40s
 11450K .......... .......... .......... .......... ..........  4%  102M 40s
 11500K .......... .......... .......... .......... ..........  4% 57.3M 40s
 11550K .......... .......... .......... .......... ..........  4%  121M 40s
 11600K .......... .......... .......... .......... ..........  4% 73.5M 39s
 11650K .......... .......... .......... .......... ..........  4% 36.3M 39s
 11700K .......... .......... .......... .......... ..........  4%  156M 39s

  6700K .......... .......... .......... .......... ..........  2%  144M 85s
  6750K .......... .......... .......... .......... ..........  2%  181M 84s
  6800K .......... .......... .......... .......... ..........  2%  151M 84s
  6850K .......... .......... .......... .......... .......... ..........  5%  561K 36s
 13550K .......... .......... .......... .......... ..........  5% 61.1M 36s
 13600K .......... .......... .......... .......... ..........  5% 97.3M 36s
 13650K .......... .......... .......... .......... ..........  5%  116M 36s
 13700K .......... .......... .......... .......... ..........  5%  100M 36s
 13750K .......... .......... .......... .......... ..........  5% 59.2M 36s
 13800K .......... .......... .......... .......... ..........  5%  129M 36s
 13850K .......... .......... .......... .......... ..........  5% 94.7M 36s
 13900K .......... .......... .......... .......... ..........  5%  165M 35s
 13950K .......... .......... .......... .......... ..........  5

...

102800K .......... .......... .......... .......... .......... 39% 86.8M 11s
102850K .......... .......... .......... .......... .......... 40% 92.1M 11s
102900K .......... .......... .......... .......... .......... 40% 72.6M 11s
102950K .......... .......... .......... .......... .......... 40%  106M 11s
103000K .......... .......... .......... .......... .......... 40% 73.8M 11s
103050K .......... .......... .......... .......... .......... 40%  129M 11s
103100K .......... .......... .......... .......... .......... 40% 99.0M 11s
103150K .......... .......... .......... .......... .......... 40% 78.5M 11s
103200K .......... .......... .......... .......... .......... 40% 64.5M 11s
103250K .......... .......... .......... .......... .......... 40% 69.2M 11s
103300K .......... .......... .......... .......... .......... 40% 76.0M 11s
103350K .......... .......... .......... .......... .......... 40% 62.7M 11s
103400K .......... .......... .......... .......... .......... 40%  109M 11s

105400K .......... .......... .......... .......... .......... 40%  147M 10s
105450K .......... .......... .......... .......... .......... 41% 57.4M 10s
105500K .......... .......... .......... .......... .......... 41%  123M 10s
105550K .......... .......... .......... .......... .......... 41%  107M 10s
105600K .......... .......... .......... .......... .......... 41% 83.2M 10s
105650K .......... .......... .......... .......... .......... 41%  109M 10s
105700K .......... .......... .......... .......... .......... 41% 77.1M 10s
105750K .......... .......... .......... .......... .... .......... 34% 1011K 14s
101900K .......... .......... .......... .......... .......... 34%  104M 14s
101950K .......... ................ 41% 41.5M 10s
105800K .......... .......... .......... ............ .......... .......... .......... 34% 67.3M 14s
102000K .......... ...... .......... 41% 43.7M 10s
105850K .......... .......... ...... .......... .......... ................ .......... .......... 41

104700K .......... .......... .......... .......... .......... 35%  169M 13s
104750K .......... .......... .......... .......... .......... 35% 89.2M 13s
104800K .......... .......... .......... .......... .......... 35% 97.1M 13s
104850K .......... .......... .......... .......... .......... 35% 51.4M 13s
104900K .......... .......... .............. .......... .......... .......... .......... 41% 1.48M 10s
107950K .......... .......... .......... .......... .......... 41% 5.98M 10s
108000K .......... .......... .......... .......... .......... 42% 3.14M 10s
108050K .......... ...... .......... .......... 35% 1.06M 13s
104950K .......... .......... .......... .......... .......... 35% 51.3M 13s
105000K .......... .......... .......... .......... .......... 35%  125M 13s
105050K .......... .......... .......... .......... .......... 35% 44.8M 13s
105100K .......... .......... .......... .......... .......... 35% 41.4M 13s
105150K .......... .......... .......... .......... .......... 35

Process is interrupted.


110900K .......... .......... ........ .......... 36% 1.32M 13s
107050K .......... .......... .......... .......... .......... 36% 31.2M 13s
107100K .......... .......... .......... .......... .......... 36% 52.6M 13s
107150K .......... .......... .......... ...... .......... .......... .......... 36% 77.3M 13s


Please note that it is a common practise to split words into subwords using Byte Pair Encoding (BPE). Please refer to [this](https://github.com/awslabs/sockeye/tree/master/tutorials/wmt) tutorial if you are interested in performing BPE.

Since training on the whole dataset might take several hours/days, for this demo, let us train on the **first 10,000 lines only**. Don't run the next cell if you want to train on the complete dataset.

In [5]:
!head -n 10000 corpus.tc.en > corpus.tc.en.small
!head -n 10000 corpus.tc.de > corpus.tc.de.small

107200K .......... .......... .......... .......... .......... 36%  107M 13s
107250K .......... .......... .......... .......... .......... 36%  118M 13s
107300K .......... .......... .......... .......... .......... 36%  106M 13s
107350K .......... .......... .......... .......... .......... 36%  104M 13s
107400K .......... .......... .......... .......... .......... 36%  100M 13s
107450K .......... .......... .......... .......... .......... 36%  114M 13s
107500K .......... .......... .......... .......... .......... 36% 96.3M 13s
107550K .......... .......... .......... .......... .......... 36%  123M 13s
107600K .......... .......... .......... .......... .......... 36% 83.8M 13s
107650K .......... .......... .......... .......... .......... 36% 76.0M 13s
107700K .......... .......... .......... .......... .......... 36%  113M 13s
107750K .......... .......... .......... .......... .......... 36%  112M 13s
107800K .......... .......... .......... .......... .......... 36%  144M 13s

head: cannot open ‘corpus.tc.en’ for reading: No such file or directory


108300K .......... .......... .......... .......... .......... 37%  155M 13s
108350K .......... .......... .......... .......... .......... 37% 93.6M 13s
108400K .......... .......... .......... .......... .......... 37%  118M 13s
108450K .......... .......... .......... .......... .......... 37% 95.9M 13s
108500K .......... .......... .......... .......... .......... 37%  122M 13s
108550K .......... .......... .......... .......... .......... 37%  109M 13s
108600K .......... .......... .......... .......... .......... 37% 59.3M 13s
108650K .......... .......... .......... .......... .......... 37%  101M 13s
108700K .......... .......... .......... .......... .......... 37% 96.1M 13s
108750K .......... .......... .......... .......... .......... 37% 41.8M 13s...... ..........
108800K .......... ...... .......... .......... .......... 43% 4.14M 10s
111150K .......... .......... .......... ........ .......... .......... .......... 37% 28.6M 13s
108850K .......... .......... .......... ..

110750K .......... .......... .......... .......... .......... 37%  113M 13s
110800K .......... .......... .......... .......... .......... 37% 67.0M 13s
110850K .......... .......... .......... .......... .......... 37%  132M 13s
110900K .......... .......... .......... .......... .......... 37% 62.0M 13s
110950K ..........
114000K .......... .......... .......... .......... .......... 44% 1.33M 10s
114050K .......... .......... .......... .......... .......... 44% 36.5M 10s
114100K .......... .. .......... .......... .......... .......... 37% 1.19M 13s
111000K .......... .......... .......... .......... .......... 37% 6.23M 13s
111050K .......... .......... .......... .......... .......... 37% 49.1M 13s
111100K .......... .......... .......... .......... .......... 37%  236M 13s
111150K .......... .......... .......... ............ .......... .......... .......... .......... 37% 24.3M 13s


head: cannot open ‘corpus.tc.de’ for reading: No such file or directory


111200K .......... .......... .......... .......... .......... 38%  121M 13s
111250K .......... .......... .......... .......... .......... 38%  154M 13s
111300K .......... .......... .......... .......... .......... 38%  158M 13s
111350K .......... .......... .......... .......... .......... 38% 23.9M 13s
111400K .......... .......... .......... .......... .......... 38%  176M 13s
111450K .......... .......... .......... .......... .......... 38%  136M 13s
111500K .......... .......... .......... .......... .......... 38%  191M 13s
111550K .......... .......... .......... .......... .......... 38%  169M 13s
111600K .......... .......... .......... .......... .......... 38%  208M 13s
111650K .......... .......... .......... .......... .......... 38%  237M 13s
111700K .......... .......... .......... .......... .......... 38%  170M 13s
111750K .......... .......... .......... .......... .......... 38%  147M 13s
111800K .......... .......... .......... .......... .......... 38% 19.9M 13s

Now, let's use the preprocessing script `create_vocab_proto.py` (provided with this notebook) to create vocabulary mappings (strings to integers) and convert these files to x-recordio-protobuf as required for training by SageMaker Seq2Seq.  
Uncomment the cell below and run to see check the arguments this script expects.

In [6]:
%%bash
# python3 create_vocab_proto.py -h

113400K .......... .......... .......... .......... .......... 38%  111M 13s
113450K .......... .......... .......... .......... .......... 38%  118M 13s
113500K .......... ............ .......... .......... .......... .......... .......... .......... 38% 53.0M 13s
113550K .......... .......... .......... .......... .......... 38%  134M 13s
113600K .......... .......... .......... .......... .......... 38% 79.5M 13s
113650K .......... .......... .......... .......... .......... 38%  107M 13s
113700K .......... .......... .......... .......... .......... 38%  104M 13s
113750K .......... .......... .......... .......... .......... 38%  126M 12s
113800K .......... .......... .......... .......... .......... 45% 1.46M 9s
117050K .......... .......... .......... .......... .......... 45% 2.14M 9s


The cell below does the preprocessing. If you are using the complete dataset, the script might take around 10-15 min on an m4.xlarge notebook instance. Remove ".small" from the file names for training on full datasets.

In [7]:
%%time
%%bash
python3 create_vocab_proto.py \
        --train-source corpus.tc.en.small \
        --train-target corpus.tc.de.small \
        --val-source validation/newstest2014.tc.en \
        --val-target validation/newstest2014.tc.de

117100K .......... .......... .......... ...... .......... 38% 1009K 13s
113850K .......... .......... .......... .......... .......... 38%  104M 13s
113900K .......... .......... .......... .......... .......... 38%  147M 13s
113950K .......... .......... .......... .......... .......... 38%  114M 13s.... .......... 45% 2.79M 9s
117150K .......... .......... .......... ....
114000K .......... .......... .......... .......... .......... 38% 47.2M 13s
114050K .......... .......... .......... .......... .......... 38%  114M 13s
114100K .......... .......... .......... .......... .......... 39%  110M 13s
114150K .......... .......... .......... .......... .......... 39% 95.2M 13s
114200K .......... .......... .......... .......... .......... 39% 88.0M 13s
114250K .......... .......... .......... .......... .......... 39%  122M 12s
114300K .......... .......... .......... .......... .......... 39% 88.5M 12s
114350K .......... .......... .......... .......... .......... 39%  127M 12s
114400

116300K .......... .......... .......... .......... .......... 39%  102M 12s
116350K .......... .......... .......... .......... .......... 39% 97.6M 12s
116400K .......... .......... .......... .......... .......... 39% 86.6M 12s
116450K .......... .......... .......... .......... .......... 39% 91.7M 12s
116500K .......... .......... .......... 46% 2.24M 9s
120050K .......... .......... ................ .......... .......... ........ .......... .......... 39% 52.5M 12s
116550K .......... .......... .......... .......... .......... 39% 88.6M 12s
116600K .......... .......... .......... .......... .......... 39% 84.5M 12s
116650K .......... .......... .......... .......... .......... 39% 98.2M 12s
116700K .......... .......... .......... .......... .......... 39%  113M 12s
116750K .......... .......... .......... .......... .......... 39% 89.0M 12s
116800K .......... .......... .......... .......... .......... 39%  102M 12s
116850K .......... .......... .......... .......... ..........

122750K .......... .......... .......... .......... .......... 47%  164M 9s
122800K .......... .......... .......... .......... .......... 47% 10.8M 9s
122850K .......... .......... .......... .......... .......... 47% 32.0M 9s
122900K .......... .......... .......... .......... .......... 47% 95.7M 9s
122950K .......... .......... .......... .......... .......... 47% 20.3M 9s
123000K .......... .......... .......... .......... .......... 47% 65.4M 9s
123050K .......... .......... .......... .............. 40% 1.61M 12s
118950K .......... .......... .......... .......... .......... 40% 45.5M 12s
119000K .......... .......... .......... .......... .......... 40%  137M 12s
119050K .......... .......... .......... .......... .......... 40% 71.8M 12s
119100K .......... .......... .......... .......... .......... 40% 59.8M 12s
119150K .......... .......... .......... .......... .......... 40%  156M 12s
119200K .......... .......... .......... .......... .......... 40% 72.8M 12s
119250K ....

125350K .......... .......... .......... .......... .......... 48% 95.4M 9s
125400K .......... .......... .......... .......... .......... 48% 85.8M 9s
125450K .......... .......... .......... .......... .......... 48% 94.1M 9s
125500K .......... .......... .......... .......... .......... 48%  101M 9s
125550K .......... .......... .......... .......... .......... 48% 96.0M 9s
125600K .......... .......... .......... .......... .......... 48% 47.4M 9s
125650K .......... .......... .......... .......... .......... 48% 53.4M 9s
125700K .......... .......... .......... .......... .......... 48% 17.3M 9s
125750K .......... .......... .......... .......... .......... 48% 42.6M 9s
125800K .......... .......... .......... .......... .......... 48% 41.2M 9s
125850K .......... .......... .......... .......... .......... 48% 12.6M 9s
125900K .......... .......... .......... .......... .......... 48% 61.2M 9s
125950K .......... .......... .......... .......... .......... 48% 93.7M 9s
126000K ....

127650K .......... .......... .......... .......... .......... 49% 97.1M 9s
127700K .......... .......... .......... .......... .......... 49% 63.6M 9s
127750K .......... .......... .......... .......... .......... 49% 64.8M 9s
127800K .......... .......... .......... .......... .......... 49% 55.5M 9s
127850K .......... .......... .......... .......... .......... 49% 29.6M 9s
127900K .......... .......... .......... .......... .......... 49% 41.8M 9s
127950K .......... .......... .......... .......... .......... 49% 45.9M 9s
128000K .......... .......... .......... .......... .......... 49% 48.3M 9s
128050K .......... .......... .......... .......... .......... 49% 46.7M 9s
128100K .......... .......... .......... .......... .......... 49% 26.9M 9s
128150K .......... .......... .......... .......... .......... 49% 7.12M 9s
128200K .......... .......... .......... .......... .......... 49%  207M 9s
128250K .......... .......... .......... .......... .......... 49%  172M 9s
128300K ....

...

208850K .......... .......... .......... .......... .......... 71% 93.3M 5s
208900K .......... ..
210000K .......... .......... .......... .......... .......... 81% 8.39M 3s
210050K .......... .......... .......... .......... .......... 81% 96.0M 3s
210100K .......... .......... .......... .......... .......... 81% 29.6M 3s
210150K .......... .......... .......... .......... .......... 81% 89.8M 3s
210200K .......... .......... .......... .......... .......... 81%  123M 3s
210250K .............. .......... .......... .......... 71% 11.0M 5s
208950K .......... .......... .......... .......... .......... 71% 48.4M 5s
209000K .......... .......... .......... .............. .......... .......... .......... .......... 81% 17.4M 3s
210300K .......... .......... .......... 71% 29.3M 5s
209050K .......... .......... .......... .......... .......... 71%  113M 5s
209100K .......... .......... .......... .......... .......... 71% 65.9M 5s
209150K .......... .......... .......... .......... ......

212700K .......... .......... .......... .......... .......... 82% 10.0M 3s
212750K .......... .......... .......... .......... .......... 82% 4.14M 3s
212800K .......... .......... .......... .......... .......... 82% 34.3M 3s
212850K .......... .......... .......... .......... .......... 82% 3.85M 3s
212900K .......... .......... .......... .......... .......... 82% 28.3M 3s
212950K .......... .......... .......... .......... .... .......... ............ 82% 46.2M 3s
213000K .............. .......... 72% 1003K 5s
211600K .......... .......... .......... .......... .......... 72%  120M 5s
211650K .......... .......... .......... .......... .......... 72%  100M 5s
211700K .......... .......... .......... .......... .......... 72%  110M 5s
211750K .......... .......... .......... .......... .......... 72% 85.0M 5s
211800K .......... .......... .......... .......... .......... 72%  147M 5s
211850K .......... .......... .......... .......... .......... 72%  113M 5s
211900K .......... ....

214050K .......... .......... .......... .......... .......... 73%  103M 5s
214100K .......... .......... .......... .......... .......... 73%  111M 5s
214150K .......... .......... .......... .......... .......... 73% 73.0M 5s
214200K .......... .......... .......... .......... .......... 73%  125M 5s
214250K .......... .......... .......... .......... .......... 73% 93.6M 5s
214300K .......... .......... .......... .......... .......... 73%  125M 5s
214350K .......... .......... .......... .......... .......... 73%  117M 5s
214400K .......... .......... .......... .......... .......... 73% 59.3M 5s
214450K .......... .......... .......... .......... .......... 73% 87.9M 5s
214500K .......... .......... .......... .......... ............ .......... 83% 2.24M 3s
215650K .......... .......... .......... .......... .......... 83% 4.58M 3s
215700K .......... .......... .......... .......... .......... 83% 25.2M 3s
215750K .......... .......... .......... .......... .......... 83% 3.68M 3s

216550K .......... .......... .......... .......... .......... 74% 37.6M 5s
216600K .......... .......... .......... .......... .......... 74% 46.5M 5s
216650K .......... .......... .......... .......... .......... .......... 84% 2.30M 2s
218550K .......... .......... .......... .......... .......... 84% 11.8M 2s
218600K .......... .......... .......... .......... .......... 85% 5.64M 2s
218650K .......... .......... .... .......... 74% 1.37M 5s
216700K .......... .......... .......... .......... .......... 74% 85.5M 5s
216750K .......... .......... .......... .......... .......... 74% 45.7M 5s
216800K .......... .......... .......... .......... .......... 74% 73.0M 5s
216850K .......... .......... .......... .......... .......... 74% 34.5M 5s
216900K .......... .......... .......... .......... .......... 74% 75.6M 5s
216950K .......... .......... .......... .......... .......... 74%  176M 5s
217000K .......... .......... .......... .......... .......... 74% 29.0M 5s
217050K ..........

219350K .......... .......... ............ .......... 85% 32.9M 2s
221200K .......... .......... .......... .......... .......... 86%  102M 2s
221250K .......... .......... .............. .......... .......... 74% 34.6M 5s
219400K .......... .......... .......... .......... .......... 74% 87.7M 5s
219450K .......... .......... .......... .......... .......... 75%  101M 5s
219500K .... .......... .......... 86% 28.4M 2s
221300K .......... .......... .......... .......... .......... 86%  113M 2s
221350K .......... .......... ............ .......... .......... .......... .......... 75% 39.2M 5s
219550K .......... ............ .......... .......... 86% 37.6M 2s
221400K .......... .......... .......... .......... .......... 86% 81.7M 2s
221450K .......... .......... .......... .......... .......... 86%  186M 2s
221500K .......... .......... .......... .......... .......... 86%  106M 2s
221550K .......... .......... .......... .......... .......... 86% 27.7M 2s
221600K .......... .......... 

The script will output 4 files, namely:
- train.rec : Contains source and target sentences for training in protobuf format
- val.rec : Contains source and target sentences for validation in protobuf format
- vocab.src.json : Vocabulary mapping (string to int) for source language (English in this example)
- vocab.trg.json : Vocabulary mapping (string to int) for target language (German in this example)

Let's upload the pre-processed dataset and vocabularies to S3

In [8]:
def upload_to_s3(bucket, prefix, channel, file):
    s3 = boto3.resource("s3")
    data = open(file, "rb")
    key = prefix + "/" + channel + "/" + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)


upload_to_s3(bucket, prefix, "train", "train.rec")
upload_to_s3(bucket, prefix, "validation", "val.rec")
upload_to_s3(bucket, prefix, "vocab", "vocab.src.json")
upload_to_s3(bucket, prefix, "vocab", "vocab.trg.json")

In [9]:
region_name = boto3.Session().region_name

In [10]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(region_name, "seq2seq")

print("Using SageMaker Seq2Seq container: {} ({})".format(container, region_name))

## Training the Machine Translation model

In [ ]:
job_name = "DEMO-seq2seq-en-de-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/".format(bucket, prefix)},
    "ResourceConfig": {
        # Seq2Seq does not support multiple machines. Currently, it only supports single machine, multiple GPUs
        "InstanceCount": 1,
        "InstanceType": "ml.p2.xlarge",  # We suggest one of ["ml.p2.16xlarge", "ml.p2.8xlarge", "ml.p2.xlarge"]
        "VolumeSizeInGB": 50,
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        # Please refer to the documentation for complete list of parameters
        "max_seq_len_source": "60",
        "max_seq_len_target": "60",
        "optimized_metric": "bleu",
        "batch_size": "64",  # Please use a larger batch size (256 or 512) if using ml.p2.8xlarge or ml.p2.16xlarge
        "checkpoint_frequency_num_batches": "1000",
        "rnn_num_hidden": "512",
        "num_layers_encoder": "1",
        "num_layers_decoder": "1",
        "num_embed_source": "512",
        "num_embed_target": "512",
        "checkpoint_threshold": "3",
        "max_num_batches": "2100"
        # Training will stop after 2100 iterations/batches.
        # This is just for demo purposes. Remove the above parameter if you want a better model.
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 48 * 3600},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
        {
            "ChannelName": "vocab",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/vocab/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
        },
    ],
}

sagemaker_client = boto3.Session().client(service_name="sagemaker")
sagemaker_client.create_training_job(**create_training_params)

status = sagemaker_client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)

In [18]:
import time

status = sagemaker_client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]

while status == "InProgress":
    time.sleep(60)
    status = sagemaker_client.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]


print(status)
# if the job failed, determine why
if status == "Failed":
    message = sagemaker_client.describe_training_job(TrainingJobName=job_name)["FailureReason"]
    print("Training failed with the following error: {}".format(message))
    raise Exception("Training job failed")

NameError: name 'sagemaker_client' is not defined

> Now wait for the training job to complete and proceed to the next step after you see model artifacts in your S3 bucket.

You can jump to [Use a pretrained model](#Use-a-pretrained-model) as training might take some time.

## Inference

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means translating sentence(s) from English to German.
This section involves several steps,
- Create model - Create a model using the artifact (model.tar.gz) produced by training
- Create Endpoint Configuration - Create a configuration defining an endpoint, using the above model
- Create Endpoint - Use the configuration to create an inference endpoint.
- Perform Inference - Perform inference on some input data using the endpoint.

### Create model
We now create a SageMaker Model from the training output. Using the model, we can then create an Endpoint Configuration.

In [15]:
use_pretrained_model = False

### Use a pretrained model
#### Please uncomment and run the cell below if you want to use a pretrained model, as training might take several hours/days to complete.

In [16]:
# use_pretrained_model = True
# model_name = "DEMO-pretrained-en-de-model"
# s3 = boto3.client("s3")
# s3.download_file(f"sagemaker-sample-files", "models/seq2seq-data/model.tar.gz", "model.tar.gz")
# s3.download_file(f"sagemaker-sample-files", "models/seq2seq-data/vocab.src.json", "vocab.src.json")
# s3.download_file(f"sagemaker-sample-files", "models/seq2seq-data/vocab.trg.json", "vocab.trg.json")
# upload_to_s3(bucket, prefix, 'pretrained_model', 'model.tar.gz')
# model_data = "s3://{}/{}/pretrained_model/model.tar.gz".format(bucket, prefix)

In [17]:
%%time

sage = boto3.client("sagemaker")

if not use_pretrained_model:
    info = sage.describe_training_job(TrainingJobName=job_name)
    model_name = job_name
    model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

print(model_name)
print(model_data)

primary_container = {"Image": container, "ModelDataUrl": model_data}

create_model_response = sage.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

NameError: name 'job_name' is not defined

### Create endpoint configuration
Use the model to create an endpoint configuration. The endpoint configuration also contains information about the type and number of EC2 instances to use when hosting the model.

Since SageMaker Seq2Seq is based on Neural Nets, we could use an ml.p2.xlarge (GPU) instance, but for this example we will use a free tier eligible ml.m4.xlarge.

In [13]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-Seq2SeqEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-Seq2SeqEndpointConfig-2023-06-08-07-22-16


NameError: name 'model_name' is not defined

### Create endpoint
Lastly, we create the endpoint that serves up model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 10-15 minutes to complete.

In [ ]:
%%time
import time

endpoint_name = "DEMO-Seq2SeqEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sage.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sage.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

# wait until the status has changed
sage.get_waiter("endpoint_in_service").wait(EndpointName=endpoint_name)

# print the status of the endpoint
endpoint_response = sage.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response["EndpointStatus"]
print("Endpoint creation ended with EndpointStatus = {}".format(status))

if status != "InService":
    raise Exception("Endpoint creation failed.")

If you see the message,
> Endpoint creation ended with EndpointStatus = InService

then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.  

We will finally create a runtime object from which we can invoke the endpoint.

In [ ]:
runtime = boto3.client(service_name="runtime.sagemaker")

## Perform Inference

### Using JSON format for inference (Suggested for a single or small number of data instances)

#### Note that you don't have to convert string to text using the vocabulary mapping for inference using JSON mode

In [ ]:
sentences = ["you are so good !", "can you drive a car ?", "i want to watch a movie ."]

payload = {"instances": []}
for sent in sentences:
    payload["instances"].append({"data": sent})

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

response = response["Body"].read().decode("utf-8")
response = json.loads(response)
print(response)

### Retrieving the Attention Matrix

Passing `"attention_matrix":"true"` in `configuration` of the data instance will return the attention matrix.

In [ ]:
sentence = "can you drive a car ?"

payload = {"instances": [{"data": sentence, "configuration": {"attention_matrix": "true"}}]}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload)
)

response = response["Body"].read().decode("utf-8")
response = json.loads(response)["predictions"][0]

source = sentence
target = response["target"]
attention_matrix = np.array(response["matrix"])

print("Source: %s \nTarget: %s" % (source, target))

In [ ]:
# Define a function for plotting the attentioan matrix
def plot_matrix(attention_matrix, target, source):
    source_tokens = source.split()
    target_tokens = target.split()
    assert attention_matrix.shape[0] == len(target_tokens)
    plt.imshow(attention_matrix.transpose(), interpolation="nearest", cmap="Greys")
    plt.xlabel("target")
    plt.ylabel("source")
    plt.gca().set_xticks([i for i in range(0, len(target_tokens))])
    plt.gca().set_yticks([i for i in range(0, len(source_tokens))])
    plt.gca().set_xticklabels(target_tokens)
    plt.gca().set_yticklabels(source_tokens)
    plt.tight_layout()

In [ ]:
plot_matrix(attention_matrix, target, source)

### Using Protobuf format for inference (Suggested for efficient bulk inference)

Reading the vocabulary mappings as this mode of inference accepts list of integers and returns list of integers.

In [ ]:
import io
import tempfile
from record_pb2 import Record
from create_vocab_proto import (
    vocab_from_json,
    reverse_vocab,
    write_recordio,
    list_to_record_bytes,
    read_next,
)

source = vocab_from_json("vocab.src.json")
target = vocab_from_json("vocab.trg.json")

source_rev = reverse_vocab(source)
target_rev = reverse_vocab(target)

In [ ]:
sentences = [
    "this is so cool",
    "i am having dinner .",
    "i am sitting in an aeroplane .",
    "come let us go for a long drive .",
]

Converting the string to integers, followed by protobuf encoding:

In [ ]:
# Convert strings to integers using source vocab mapping. Out-of-vocabulary strings are mapped to 1 - the mapping for <unk>
sentences = [[source.get(token, 1) for token in sentence.split()] for sentence in sentences]
f = io.BytesIO()
for sentence in sentences:
    record = list_to_record_bytes(sentence, [])
    write_recordio(f, record)

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/x-recordio-protobuf", Body=f.getvalue()
)

response = response["Body"].read()

Now, parse the protobuf response and convert list of integers back to strings

In [ ]:
def _parse_proto_response(received_bytes):
    output_file = tempfile.NamedTemporaryFile()
    output_file.write(received_bytes)
    output_file.flush()
    target_sentences = []
    with open(output_file.name, "rb") as datum:
        next_record = True
        while next_record:
            next_record = read_next(datum)
            if next_record:
                rec = Record()
                rec.ParseFromString(next_record)
                target = list(rec.features["target"].int32_tensor.values)
                target_sentences.append(target)
            else:
                break
    return target_sentences

In [ ]:
targets = _parse_proto_response(response)
resp = [" ".join([target_rev.get(token, "<unk>") for token in sentence]) for sentence in targets]
print(resp)

## Stop / Close the Endpoint (Optional)

Finally, we should delete the endpoint before we close the notebook.

In [ ]:
sage.delete_endpoint(EndpointName=endpoint_name)